# Dataset Preperation Visualization

Use this code to ineractively view the different stages the data went through.

In [ ]:
import scipy
from IPython.display import Image, display
import matplotlib.pyplot as plt
import trimesh
import json
import os
import random
import pandas as pd

In [ ]:
# load config
config_path = "../config.json"
config = json.load(open(config_path, "r"))

In [ ]:
# load dataset
data = pd.read_csv(os.path.join(config["paths"]["datasets"], "test.csv"))
data.head()

In [ ]:
# get random dataset_id
dataset_id = random.choice(data["dataset_id"].tolist())
dataset_id

In [ ]:
# Setting up paths from config and chosen dataset_id

model_result_path = os.path.join(
    config["paths"]["checkpoints"],
    config["face_geometry"]["model"],
    "results",
    f"epoch_{config['face_geometry']['epoch']:02d}_000000",
)
bfm_path = config["paths"]["bfm"]

image_file = os.path.join(model_result_path, f"{dataset_id}_M.png")
coefficient_path = os.path.join(model_result_path, f"{dataset_id}_M.mat")
obj_path = os.path.join(model_result_path, f"{dataset_id}_M.obj")

In [ ]:
display(Image(filename = image_file))

In [ ]:
mat_data = scipy.io.loadmat(coefficient_path)

mesh = trimesh.load(obj_path)
vertices_3d = mesh.vertices

bfm_model_front = scipy.io.loadmat(os.path.join(bfm_path, 'BFM_model_front.mat'))

landmark_indices = bfm_model_front['keypoints'].flatten() - 1
landmark_vertices_3d = vertices_3d[landmark_indices.astype(int)]
print(f"3D landmarks shape: {landmark_vertices_3d.shape}")

In [ ]:
# Define facial landmark connections (typical 68-point connections)
connections = [
    # Jaw line (0-16)
    list(range(0, 17)),
    # Right eyebrow (17-21)
    list(range(17, 22)),
    # Left eyebrow (22-26)
    list(range(22, 27)),
    # Nose bridge (27-30)
    list(range(27, 31)),
    # Nose base (31-35)
    list(range(31, 36)),
    # Right eye (36-41)
    list(range(36, 42)) + [36],
    # Left eye (42-47)
    list(range(42, 48)) + [42],
    # Outer mouth (48-59)
    list(range(48, 60)) + [48],
    # Inner mouth (60-67)
    list(range(60, 68)) + [60]
]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(landmark_vertices_3d[:, 0], landmark_vertices_3d[:, 1], landmark_vertices_3d[:, 2], 
           c='red', s=20)

for connection in connections:
    ax.plot(landmark_vertices_3d[connection, 0], 
            landmark_vertices_3d[connection, 1], 
            landmark_vertices_3d[connection, 2], 
            'b-', linewidth=1)

for i, (x, y, z) in enumerate(landmark_vertices_3d):
    ax.text(x, y, z, str(i), fontsize=8, ha='center', va='center', 
            bbox=dict(boxstyle="round,pad=0.1", facecolor="white", alpha=0.7))

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Facial Landmarks with Point Indices')
ax.view_init(elev=90, azim=-90)
plt.show()